In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("SuperMarket.csv")

In [ ]:
df.info()

In [ ]:
print(df.isnull().sum())

In [ ]:
segment_counts = df['Segment'].value_counts()
print(segment_counts)

In [ ]:
country_counts = df['Country/Region'].value_counts()
print(country_counts)

In [ ]:
city_counts = df['City'].value_counts() 
print(city_counts)

In [ ]:
Danbury_order = df[df['City'] == 'Danbury'].to_string(index=False)
print(Danbury_order)

In [ ]:
postal_code_count = df['Postal Code'].value_counts()
print(postal_code_count)

In [ ]:
postal = df[df['Postal Code']== '10035']
print(postal)

In [ ]:
print(df['Category'].value_counts())

In [12]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d-%m-%Y')  # Specify the format
df['Day'] = df['Order Date'].dt.day
df['Month'] = df['Order Date'].dt.month
df['Year'] = df['Order Date'].dt.year
# print(df['Day'].head())
# print(df['Month'].head())
# print(df['Year'].head())

In [ ]:
print(df[df['Sales']  > 600])

In [ ]:
print(df['Sales'].max())

Descriptive Analysis

In [ ]:
# Total and average sales by region, category, and segment
region_sales = df.groupby('Region')['Sales'].agg(['sum', 'mean']).reset_index()
category_sales = df.groupby('Category')['Sales'].agg(['sum', 'mean']).reset_index()
segment_sales = df.groupby('Segment')['Sales'].agg(['sum', 'mean']).reset_index()

# Convert Order Date to datetime for trend analysis
df['Order Date'] = pd.to_datetime(df['Order Date'])

# Group by month and calculate total sales and profit
trend_data = df.resample('M', on='Order Date').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()

region_sales, category_sales, segment_sales, trend_data.head()

Customer Segmentation: Use clustering (e.g., K-Means) to group customers based on purchase behavior.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Prepare the data for clustering: Group by Customer ID and calculate aggregated metrics
customer_data = df.groupby('Customer ID').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum'
}).reset_index()

# Standardize the data for clustering
scaler = StandardScaler()
scaled_data = scaler.fit_transform(customer_data[['Sales', 'Profit', 'Quantity']])

# Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
customer_data['Cluster'] = kmeans.fit_predict(scaled_data)

# Show resulting clusters and the first few rows
customer_clusters = customer_data.groupby('Cluster').agg({
    'Sales': ['mean', 'sum'],
    'Profit': ['mean', 'sum'],
    'Quantity': ['mean', 'sum']
}).reset_index()

customer_data, customer_clusters

In [ ]:
cus_10315 = df[df['Customer ID'] == 'AA-10315']
# print(cus_10315.head())
cus_10315['Sales'].sum()

In [ ]:
df['Sales'].sum()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Group by month and aggregate sales data
monthly_sales = df.groupby(df['Order Date'].dt.to_period('M'))['Sales'].sum()

# Ensure 'Order Date' is in datetime format
df['Order Date'] = pd.to_datetime(df['Order Date'])

# Group by month and aggregate sales data (ensure proper datetime index)
monthly_sales = df.groupby(df['Order Date'].dt.to_period('M'))['Sales'].sum()

# Convert PeriodIndex to datetime for proper handling by ARIMA or Prophet
monthly_sales.index = monthly_sales.index.to_timestamp()

# Fit ARIMA model (p, d, q values can be adjusted)
model = ARIMA(monthly_sales, order=(5, 1, 0))  # p, d, q can be tuned
model_fit = model.fit()

# Make predictions for the next 12 months
forecast = model_fit.forecast(steps=12)

plt.plot(monthly_sales.index, monthly_sales.values, label="Actual Sales")
plt.plot(pd.date_range(monthly_sales.index[-1], periods=13, freq='M')[1:], forecast, label="Forecasted Sales", color='red')
plt.legend()
plt.show()

Profitability Analysis : Identify the most/least profitable products, customers, and regions.

In [ ]:
# Most and Least Profitable Products

# Calculate total profit for each product
product_profit = df.groupby('Product Name').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()

# Sort by profit to find the most and least profitable products
product_profit_sorted = product_profit.sort_values(by='Profit', ascending=False)

# Display the top and bottom products
most_profitable_product = product_profit_sorted.head(10)
least_profitable_product = product_profit_sorted.tail(10)

print("Most Profitable Products:")
print(most_profitable_product)
print("\nLeast Profitable Products:")
print(least_profitable_product)

In [ ]:
df[df['Product Name'] == 'Canon imageCLASS 2200 Advanced Copier']['Sales'].sum()

In [ ]:
# Most and Least Profitable Customers

# Calculate total profit for each customer
customer_profit = df.groupby('Customer ID').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()

# Sort by profit to find the most and least profitable customers
customer_profit_sorted = customer_profit.sort_values(by='Profit', ascending=False)

# Display the top and bottom customers
most_profitable_customer = customer_profit_sorted.head(10)
least_profitable_customer = customer_profit_sorted.tail(10)

print("Most Profitable Customers:")
print(most_profitable_customer)
print("\nLeast Profitable Customers:")
print(least_profitable_customer)

In [ ]:
customer_data = df[df['Customer ID'] == 'TC-20980']
print(f"Sales sum = {customer_data['Sales'].sum()}, Profit sum = {customer_data['Profit'].sum()}")

In [ ]:
# Most and Least Profitable Regions

# Calculate total profit for each region
region_profit = df.groupby('Region').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()

# Sort by profit to find the most and least profitable regions
region_profit_sorted = region_profit.sort_values(by='Profit', ascending=False)

# Display the top and bottom regions
most_profitable_region = region_profit_sorted.head(10)
least_profitable_region = region_profit_sorted.tail(10)

print("Most Profitable Regions:")
print(most_profitable_region)
print("\nLeast Profitable Regions:")
print(least_profitable_region)

In [ ]:
print("Sales = " , df[df['Region'] == 'West']['Sales'].sum())
print("Profit = " , df[df['Region'] == 'West']['Profit'].sum())

Discount Optimization: Assess the impact of discounts on sales and profit to suggest optimal discount strategies.

In [ ]:
# If not, you can calculate it as: (List Price - Sale Price) / List Price
df['Discount Percentage'] = df['Discount'] * 100  # Assuming 'Discount' is a proportion (e.g., 0.1 for 10%)

# Create discount ranges
df['Discount Range'] = pd.cut(df['Discount Percentage'], bins=[0, 5, 10, 15, 20, 25, 30, 100], 
                              labels=["0-5%", "5-10%", "10-15%", "15-20%", "20-25%", "25-30%", "30%+"])

# Calculate average sales and profit for each discount range
discount_analysis = df.groupby('Discount Range').agg({'Sales': 'mean', 'Profit': 'mean'}).reset_index()

print(discount_analysis)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Step 3: Plot the relationship between discount and average sales/profit
plt.figure(figsize=(10, 6))

# Plot average sales and profit by discount range
sns.barplot(x='Discount Range', y='Sales', data=discount_analysis, color='blue', alpha=0.6, label='Sales')
sns.barplot(x='Discount Range', y='Profit', data=discount_analysis, color='green', alpha=0.6, label='Profit')

plt.title("Impact of Discounts on Sales and Profit")
plt.xlabel("Discount Range (%)")
plt.ylabel("Value")
plt.legend()
plt.show()

In [ ]:
# Calculate correlation between Discount, Sales, and Profit
correlation_matrix = df[['Discount Percentage', 'Sales', 'Profit']].corr()

# Display the correlation matrix
print(correlation_matrix)

Geographical Heatmaps: Visualize sales and profit distribution geographically.

In [ ]:
# Group by geographic region
geographical_data = df.groupby('Region').agg({'Sales': 'sum', 'Profit': 'sum'}).reset_index()

print(geographical_data)

Anomaly Detection: Detect unusual trends in sales, profit, or discount using outlier detection techniques.

In [ ]:
# Select relevant columns for anomaly detection
anomaly_data = df[['Sales', 'Profit', 'Discount']]

# Check for missing values and handle them if any
print(anomaly_data.isnull().sum())

In [ ]:
from scipy.stats import zscore

# Calculate Z-scores for each column
z_scores = anomaly_data.apply(zscore)

# Define a threshold for anomalies (e.g., Z > 3 or Z < -3)
threshold = 3
anomalies = (z_scores.abs() > threshold)

# Flag and display rows with anomalies
anomalous_rows = df[anomalies.any(axis=1)]
print(anomalous_rows)

In [ ]:
from sklearn.ensemble import IsolationForest

# Train an Isolation Forest model
iso_forest = IsolationForest(contamination=0.05, random_state=42)
df['Anomaly_Score'] = iso_forest.fit_predict(anomaly_data)

# Identify anomalies (label -1 as anomalies)
anomalies = df[df['Anomaly_Score'] == -1]
print(anomalies)

In [ ]:
from sklearn.cluster import DBSCAN

# Fit DBSCAN on the data
dbscan = DBSCAN(eps=0.5, min_samples=5)
labels = dbscan.fit_predict(anomaly_data)

# Add labels to the dataframe
df['Cluster'] = labels

# Identify outliers (label -1 as anomalies)
outliers = df[df['Cluster'] == -1]
print(outliers)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Scatter plot to visualize anomalies
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Sales', y='Profit', hue=(df['Anomaly_Score'] == -1), palette=['blue', 'red'])
plt.title('Anomaly Detection in Sales and Profit')
plt.legend(['Normal', 'Anomaly'])
plt.show()


Shipping Efficiency: Analyze shipping times and modes to optimize delivery schedules.

In [ ]:
# Convert Order Date and Ship Date to datetime format with dayfirst=True
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True, errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], dayfirst=True, errors='coerce')

# Check for any NaT (missing dates due to parsing issues)
print(df[['Order Date', 'Ship Date']].isna().sum())

# Calculate Delivery Time in days
df['Delivery Time'] = (df['Ship Date'] - df['Order Date']).dt.days

# Verify the result
print(df[['Order Date', 'Ship Date', 'Delivery Time']].head())

In [ ]:
# Group by Shipping Mode and calculate delivery statistics
shipping_stats = df.groupby('Ship Mode').agg({
    'Delivery Time': ['mean', 'median', 'max', 'std']
}).reset_index()

print(shipping_stats)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Boxplot of Delivery Time by Shipping Mode
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='Ship Mode', y='Delivery Time', palette='Set2')
plt.title('Delivery Time by Shipping Mode')
plt.xlabel('Shipping Mode')
plt.ylabel('Delivery Time (days)')
plt.legend()
plt.show()

In [ ]:
# Calculate thresholds for delays (e.g., > mean + 2*std)
thresholds = shipping_stats['Delivery Time']['mean'] + 2 * shipping_stats['Delivery Time']['std']
threshold_dict = dict(zip(shipping_stats['Ship Mode'], thresholds))

# Flag delayed shipments
df['Delayed'] = df.apply(
    lambda row: row['Delivery Time'] > threshold_dict[row['Ship Mode']],
    axis=1
)

# Analyze delayed shipments
delayed_orders = df[df['Delayed']]
print(delayed_orders[['Ship Mode', 'Delivery Time']].head())